# Native Hydrogen Bond Contact Analysis

This notebook calculates the percentage of native hydrogen bonds maintained throughout a molecular dynamics trajectory. The hydrogen bond criteria are based on the upside2 force field implementation in `hbond.cpp`.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Add utils to path for mdtraj_upside
sys.path.append('../ff2_rg/utils')
import mdtraj_upside as mu

%matplotlib inline

## Hydrogen Bond Definition

Based on `hbond.cpp`, hydrogen bonds are defined by:
- **Distance constraint**: H-O distance ≤ 3.5 Å
- **Angular constraints**: 
  - `dot(rHO, rOC) > 0` (where rHO is H→O vector, rOC is O-C bond direction)
  - `dot(rHO, rHN) < 0` (where rHN is H-N bond direction)

In [ ]:
# Hydrogen bond parameters from hbond.cpp
DISTANCE_CUTOFF = 0.35  # 3.5 Angstroms converted to nanometers (MDTraj units)
ANGULAR_CUTOFF = 0.0   # dot product threshold

In [ ]:
def identify_hbond_atoms(traj):
    """
    Identify hydrogen bond donors and acceptors in the trajectory.
    
    Returns:
        donors: list of (H_idx, N_idx) tuples
        acceptors: list of (O_idx, C_idx) tuples
    """
    donors = []
    acceptors = []
    
    topology = traj.topology
    
    # Find hydrogen bond donors (H bonded to N)
    for bond in topology.bonds:
        atom1, atom2 = bond
        if atom1.element.symbol == 'H' and atom2.element.symbol == 'N':
            donors.append((atom1.index, atom2.index))
        elif atom1.element.symbol == 'N' and atom2.element.symbol == 'H':
            donors.append((atom2.index, atom1.index))
    
    # Find hydrogen bond acceptors (O bonded to C)
    for bond in topology.bonds:
        atom1, atom2 = bond
        if atom1.element.symbol == 'O' and atom2.element.symbol == 'C':
            acceptors.append((atom1.index, atom2.index))
        elif atom1.element.symbol == 'C' and atom2.element.symbol == 'O':
            acceptors.append((atom2.index, atom1.index))
    
    print(f"Found {len(donors)} potential hydrogen donors")
    print(f"Found {len(acceptors)} potential hydrogen acceptors")
    
    return donors, acceptors

In [ ]:
def calculate_hbonds_frame(xyz, donors, acceptors):
    """
    Calculate hydrogen bonds for a single frame.
    
    Args:
        xyz: coordinates for the frame (n_atoms, 3) in nanometers
        donors: list of (H_idx, N_idx) tuples
        acceptors: list of (O_idx, C_idx) tuples
    
    Returns:
        hbonds: list of tuples (donor_idx, acceptor_idx, distance, angle1, angle2)
    """
    hbonds = []
    
    for i, (h_idx, n_idx) in enumerate(donors):
        for j, (o_idx, c_idx) in enumerate(acceptors):
            # Skip if same residue (simple check)
            if abs(h_idx - o_idx) < 3:  # rough same residue filter
                continue
                
            # Get coordinates (in nanometers)
            h_pos = xyz[h_idx]
            n_pos = xyz[n_idx]
            o_pos = xyz[o_idx]
            c_pos = xyz[c_idx]
            
            # Calculate H-O distance (in nanometers)
            ho_vec = o_pos - h_pos
            ho_dist = np.linalg.norm(ho_vec)
            
            # Check distance constraint (both in nanometers)
            if ho_dist > DISTANCE_CUTOFF:
                continue
            
            # Calculate unit vectors
            rHO = ho_vec / ho_dist  # H to O direction
            
            # H-N bond direction (N to H)
            hn_vec = h_pos - n_pos
            hn_norm = np.linalg.norm(hn_vec)
            if hn_norm < 1e-6:
                continue
            rHN = hn_vec / hn_norm
            
            # O-C bond direction (C to O)
            oc_vec = o_pos - c_pos
            oc_norm = np.linalg.norm(oc_vec)
            if oc_norm < 1e-6:
                continue
            rOC = oc_vec / oc_norm
            
            # Calculate angular constraints
            dotHOC = np.dot(rHO, rOC)
            dotOHN = -np.dot(rHO, rHN)  # Note the negative sign from hbond.cpp
            
            # Check angular constraints
            if dotHOC > ANGULAR_CUTOFF and dotOHN > ANGULAR_CUTOFF:
                # Store distance in Angstroms for output consistency
                hbonds.append((i, j, ho_dist * 10.0, dotHOC, dotOHN))
    
    return hbonds

In [ ]:
def create_contact_map_dataframe(hbonds, frame_idx, donors, acceptors):
    """
    Create a pandas DataFrame for hydrogen bond contact map.
    """
    if not hbonds:
        return pd.DataFrame(columns=['frame', 'donor_idx', 'acceptor_idx', 'donor_h', 'donor_n', 
                                   'acceptor_o', 'acceptor_c', 'distance', 'angle_hoc', 'angle_ohn'])
    
    data = []
    for donor_idx, acceptor_idx, distance, angle_hoc, angle_ohn in hbonds:
        h_idx, n_idx = donors[donor_idx]
        o_idx, c_idx = acceptors[acceptor_idx]
        
        data.append({
            'frame': frame_idx,
            'donor_idx': donor_idx,
            'acceptor_idx': acceptor_idx,
            'donor_h': h_idx,
            'donor_n': n_idx,
            'acceptor_o': o_idx,
            'acceptor_c': c_idx,
            'distance': distance,  # in Angstroms
            'angle_hoc': angle_hoc,
            'angle_ohn': angle_ohn
        })
    
    return pd.DataFrame(data)

In [ ]:
def calculate_native_contacts_percentage(traj_file, reference_frame=0, stride=1):
    """
    Calculate the percentage of native hydrogen bonds for each frame in trajectory.
    
    Args:
        traj_file: path to trajectory file
        reference_frame: frame to use as native reference (default: 0)
        stride: stride for trajectory reading
    
    Returns:
        results: dict with 'percentages', 'native_contacts', 'all_contacts'
    """
    print(f"Loading trajectory: {traj_file}")
    traj = mu.load_upside_traj(traj_file, stride=stride)
    print(f"Loaded {traj.n_frames} frames, {traj.n_atoms} atoms")
    
    # Identify potential hydrogen bond partners
    donors, acceptors = identify_hbond_atoms(traj)
    
    # Calculate native contacts (reference frame)
    print(f"Calculating native contacts from frame {reference_frame}...")
    native_hbonds = calculate_hbonds_frame(traj.xyz[reference_frame], donors, acceptors)
    native_contact_set = set((donor_idx, acceptor_idx) for donor_idx, acceptor_idx, _, _, _ in native_hbonds)
    
    print(f"Found {len(native_contact_set)} native hydrogen bonds")
    
    # Store native contacts DataFrame
    native_df = create_contact_map_dataframe(native_hbonds, reference_frame, donors, acceptors)
    
    # Calculate contacts for each frame
    percentages = []
    all_contacts = []
    
    print("Analyzing trajectory frames...")
    for frame_idx in range(traj.n_frames):
        if frame_idx % 100 == 0:
            print(f"Processing frame {frame_idx}/{traj.n_frames}")
            
        hbonds = calculate_hbonds_frame(traj.xyz[frame_idx], donors, acceptors)
        contact_set = set((donor_idx, acceptor_idx) for donor_idx, acceptor_idx, _, _, _ in hbonds)
        
        # Calculate percentage of native contacts present
        if len(native_contact_set) > 0:
            native_present = len(native_contact_set.intersection(contact_set))
            percentage = 100.0 * native_present / len(native_contact_set)
        else:
            percentage = 0.0
        
        percentages.append(percentage)
        
        # Store detailed contact information
        frame_df = create_contact_map_dataframe(hbonds, frame_idx, donors, acceptors)
        all_contacts.append(frame_df)
    
    print("Analysis complete!")
    
    return {
        'percentages': np.array(percentages),
        'native_contacts': native_df,
        'all_contacts': pd.concat(all_contacts, ignore_index=True) if all_contacts else pd.DataFrame(),
        'native_contact_pairs': native_contact_set,
        'donors': donors,
        'acceptors': acceptors
    }

## Example Usage

Let's analyze a trajectory file. You can modify the trajectory path below:

In [ ]:
# Example trajectory file - modify this path as needed
traj_file = "../ff2_rg/inputs/a3d.run.up"  # or any other .up trajectory file

# Check if file exists
if os.path.exists(traj_file):
    print(f"Found trajectory file: {traj_file}")
else:
    print(f"Trajectory file not found: {traj_file}")
    print("Available trajectory files:")
    for file in os.listdir("../ff2_rg/inputs/"):
        if file.endswith(".up"):
            print(f"  {file}")

In [ ]:
# Run the analysis
results = calculate_native_contacts_percentage(traj_file, stride=10)  # Use stride=10 for faster analysis

## Visualization and Analysis

In [ ]:
def plot_native_contact_percentage(percentages, stride=1):
    """
    Plot the percentage of native contacts over time.
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    frames = np.arange(len(percentages)) * stride
    
    # Time series plot
    ax1.plot(frames, percentages, linewidth=1, alpha=0.7)
    ax1.set_xlabel('Frame')
    ax1.set_ylabel('Native H-bonds (%)')
    ax1.set_title('Native Hydrogen Bond Percentage Over Time')
    ax1.grid(True, alpha=0.3)
    
    # Distribution histogram
    ax2.hist(percentages, bins=50, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Native H-bonds (%)')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Native H-bond Percentages')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Statistics:")
    print(f"  Mean: {np.mean(percentages):.2f}%")
    print(f"  Std:  {np.std(percentages):.2f}%")
    print(f"  Min:  {np.min(percentages):.2f}%")
    print(f"  Max:  {np.max(percentages):.2f}%")

In [ ]:
def analyze_contact_map(native_contacts, all_contacts):
    """
    Analyze the hydrogen bond contact map.
    """
    print("Native Hydrogen Bonds:")
    print(native_contacts[['donor_h', 'donor_n', 'acceptor_o', 'acceptor_c', 'distance', 'angle_hoc', 'angle_ohn']])
    
    if len(all_contacts) > 0:
        # Contact frequency analysis
        contact_freq = all_contacts.groupby(['donor_idx', 'acceptor_idx']).size().reset_index(name='frequency')
        contact_freq = contact_freq.sort_values('frequency', ascending=False)
        
        print("\nMost Frequent Hydrogen Bond Contacts:")
        print(contact_freq.head(10))
        
        # Distance and angle distributions
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        axes[0].hist(all_contacts['distance'], bins=30, alpha=0.7, edgecolor='black')
        axes[0].set_xlabel('H-O Distance (Å)')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title('H-O Distance Distribution')
        axes[0].axvline(3.5, color='red', linestyle='--', label='Cutoff (3.5 Å)')
        axes[0].legend()
        
        axes[1].hist(all_contacts['angle_hoc'], bins=30, alpha=0.7, edgecolor='black')
        axes[1].set_xlabel('H-O-C Angle (dot product)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('H-O-C Angular Distribution')
        axes[1].axvline(ANGULAR_CUTOFF, color='red', linestyle='--', label=f'Cutoff ({ANGULAR_CUTOFF})')
        axes[1].legend()
        
        axes[2].hist(all_contacts['angle_ohn'], bins=30, alpha=0.7, edgecolor='black')
        axes[2].set_xlabel('O-H-N Angle (dot product)')
        axes[2].set_ylabel('Frequency')
        axes[2].set_title('O-H-N Angular Distribution')
        axes[2].axvline(ANGULAR_CUTOFF, color='red', linestyle='--', label=f'Cutoff ({ANGULAR_CUTOFF})')
        axes[2].legend()
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Example analysis (uncomment after running the calculation above)
plot_native_contact_percentage(results['percentages'])
analyze_contact_map(results['native_contacts'], results['all_contacts'])

## Additional Analysis Functions

In [ ]:
def save_results(results, output_prefix):
    """
    Save analysis results to files.
    """
    # Save time series
    np.savetxt(f"{output_prefix}_native_hbond_percentage.txt", 
               results['percentages'], 
               header="Frame\tNative_HBond_Percentage")
    
    # Save contact maps
    results['native_contacts'].to_csv(f"{output_prefix}_native_contacts.csv", index=False)
    results['all_contacts'].to_csv(f"{output_prefix}_all_contacts.csv", index=False)
    
    print(f"Results saved with prefix: {output_prefix}")

def load_results(output_prefix):
    """
    Load previously saved results.
    """
    percentages = np.loadtxt(f"{output_prefix}_native_hbond_percentage.txt")
    native_contacts = pd.read_csv(f"{output_prefix}_native_contacts.csv")
    all_contacts = pd.read_csv(f"{output_prefix}_all_contacts.csv")
    
    return {
        'percentages': percentages,
        'native_contacts': native_contacts,
        'all_contacts': all_contacts
    }

## Instructions for Use

1. **Set trajectory path**: Modify the `traj_file` variable to point to your trajectory file
2. **Run analysis**: Execute the cells to run `calculate_native_contacts_percentage()`
3. **Visualize results**: Run the plotting functions
4. **Save/load**: Use the save/load functions to preserve results

The script will output:
- Time series of native hydrogen bond percentages
- Detailed contact maps in pandas DataFrames
- Statistics and visualizations

Note: The hydrogen bond criteria exactly match those used in the upside2 force field (`hbond.cpp`).

**Units Note**: MDTraj stores coordinates in nanometers, so the distance cutoff has been converted from 3.5 Å to 0.35 nm for consistency.